# Visualization Project

---
**Authors**:
-  *Juan P. Zaldivar E.*
-  *Enrique Millán X.*
---

## Introduction

This file contains the preprocessing required for the datasets. The datasets are:
- Colissions dataset.
- Weather dataset.
- New York Map.


<!-- explicar que primer se hace una exploracion visual y luego el preprocesing? -->

### Dataset obtention

The [*collision*](https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95) dataset was already given by the instructors of the project.

The *weather* dataset was obtained following the next steps:

- Visit the [NOAA Climate Data Online Search](https://www.ncdc.noaa.gov/cdo-web/search) web page.

- Select the following options:
  - `Weather Observation Type/Dataset -> Daily Summaries, Date Range -> 2018-01-01 to 2020-12-31, Search For -> Cities, Search Term -> New York City.`

- Look for "*New York, NY US*" and click in ADD TO CART. Now, click the cart in the top right corner.

- Select "*Custom GHCN-Daily CSV*", and the date previously selected (2018-01-01 to 2020-12-31). We are selecting more information than needed (to avoid disjoint downloads), but we will later filter it with ``Pandas`` and ``Open Refine``. Click continue.

- Fill the three options, and select "*metric units*".

- Fill all the options remaining and click continue. There are some options that will be probably not needed, but we will further analyze this when cleaning the datasets.

- Type the email where you want to receive the data so the order can start.

The *map* dataset was obtained following the next steps:

The datasets are located in the folder `Data/` and the results are saved in the folder `Data/Preprocessed/`. Following are the loading of each dataset and the import of the required libraries.

### Libraries

In [99]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from Modules import preprocessing as pre

In [100]:
# !pip install numpy
# comentar las versiones de los paquetes

## Dataset preprocessing

The preprocessing of the files was done conjointly with OpenRefine and the proposed python libraries in order to be able to take advantages of both tools.

In [101]:
dir = './Data'
collision_exists = False

## Collision dataset preprocessing

### Data exploration?

The collision table incorporates information pertaining to individual crash events, with each row representing a distinct collision incident. The dataframe compile details from all police reported motor vehicle collisions in NYC. 

<!-- ...se hace una breve descripcion de la exploracion? -->

The original dataset was first loaded into a ``Pandas`` dataframe in order to filter the desired range of dates. The reason lays in a more efficient way to filter the data, taking the size of the original dataset into account. This volumn of data made the computational process in OpenRefine very slow and ineffcient. After this initial filtering, the dataset was exported to a ``.csv`` file and loaded into OpenRefine.

In [102]:
if os.path.exists(f'{dir}/collisions_2018-2020.csv'):
    collision = pd.read_csv(f'{dir}/collisions_2018-2020.csv')
    colission_exists = True
else:
    collision = pd.read_csv(f'{dir}/collisions.csv')

collision.shape

(115740, 29)

In [103]:
collision.head()

,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2020-09-06,18:05,NaN,NaN,40.771038,-73.83413,"(40.771038, -73.83413)",WHITESTONE EXPRESSWAY,NaN,NaN,...,Following Too Closely,NaN,NaN,NaN,4345591,Station Wagon/Sport Utility Vehicle,Motorcycle,NaN,NaN,NaN
1,2020-09-20,9:14,NaN,NaN,40.722095,-73.77772,"(40.722095, -73.77772)",GRAND CENTRAL PKWY,NaN,NaN,...,Unspecified,NaN,NaN,NaN,4459141,Sedan,Pick-up Truck,NaN,NaN,NaN
2,2020-09-24,22:00,MANHATTAN,10039.0,40.824757,-73.94052,"(40.824757, -73.94052)",8 AVENUE,WEST 148 STREET,NaN,...,Driver Inattention/Distraction,NaN,NaN,NaN,4461437,Sedan,Motorscooter,NaN,NaN,NaN
3,2020-08-06,5:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120 Huge Grant Circle,...,Unspecified,NaN,NaN,NaN,4336560,Sedan,Station Wagon/Sport Utility Vehicle,NaN,NaN,NaN
4,2020-08-01,17:16,BRONX,10462.0,40.840508,-73.85515,"(40.840508, -73.85515)",METROPOLITAN AVENUE,LINDEN DRIVE,NaN,...,Unspecified,NaN,NaN,NaN,4334713,Sedan,E-Bike,NaN,NaN,NaN


If the filtered version does not exist, we proceed with the filtering. To filter the data to the summer of 2018 and 2020, the first step is to change the data type of the **CRASH DATE** attribute.

In [104]:
if not colission_exists:
    collision = pre.time_filter(collision, 'CRASH DATE')
    collision.to_csv(f'{dir}/collisions_2018-2020.csv', index=False)

collision.shape

(115740, 29)

After the filtering, the dataset was exported to a ``.csv`` file and loaded into OpenRefine. The procedure and reasoning taken in OpenRefine will be explained and justified in the present section.

### Data type conversion

The data conversion was done in OpenRefine. The **CRASH DATE** attribute was converted to a date type. The **COLLISION ID** and **CRASH TIME** were set as strings for the time being.

 The attributes relating the greographic location of the collision were set as strings with some special remarks. All the values were set to uppercase and the extra spaces were removed (if any). The reason for this was to ease the work of the clusterization method used to collectively inspect and edit cells in the case that there were some values that were not correctly or consistently written.

For the attributes refering to the number of persons involved in the collision, the data type was set to integer. The reason for this is that the values are discrete and the values are not expected to be negative.

The vehicle and factors attributes were left as strings for the time being. 

### Data selection and transformation

All of the following transformations were applied with OpenRefine, but some comprobations to justify the transformations are carried on in this section of the notebook.

In [105]:
precollision = pd.read_csv(f'{dir}/collisions_2018-2020_prepro_v1.csv')

#### Time attributes

In [106]:
precollision["CRASH DATE"] = pd.to_datetime(precollision["CRASH DATE"])
precollision['YEAR'] = precollision['CRASH DATE'].dt.year
precollision["DAY NAME"] = precollision["CRASH DATE"].dt.day_name()
precollision["TYPE OF DAY"] = np.where(precollision["DAY NAME"].isin(["Saturday", "Sunday"]), "Weekend", "Weekday")

for _ in range(3):
    cols = precollision.columns.tolist()
    cols = cols[:1] + cols[-1:] + cols[1:-1]
    precollision = precollision[cols]

precollision.head()

,COLLISION_ID,YEAR,DAY NAME,TYPE OF DAY,CRASH DATE,CRASH TIME,BOROUGH,ZIP CODE,LOCATION,STREET NAME,NUMBER OF INJURED,NUMBER OF KILLED,CONTRIBUTING FACTOR VEHICLE 1,VEHICLE TYPE CODE 1
0,4345591,2020,Sunday,Weekend,2020-09-06 00:00:00+00:00,18:05,NaN,NaN,"(40.771038, -73.83413)",WHITESTONE EXPRESSWAY,0.0,2.0,Unsafe Lane Changing,VAN
1,4459141,2020,Sunday,Weekend,2020-09-20 00:00:00+00:00,9:14,NaN,NaN,"(40.722095, -73.77772)",GRAND CENTRAL PKWY,0.0,0.0,Driver Inattention/Distraction,CAR
2,4461437,2020,Thursday,Weekday,2020-09-24 00:00:00+00:00,22:00,MANHATTAN,10039.0,"(40.824757, -73.94052)",8 AVENUE,0.0,0.0,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,CAR
3,4336560,2020,Thursday,Weekday,2020-08-06 00:00:00+00:00,5:30,NaN,NaN,NaN,120 HUGE GRANT CIRCLE,2.0,0.0,Unspecified,CAR
4,4334713,2020,Saturday,Weekend,2020-08-01 00:00:00+00:00,17:16,BRONX,10462.0,"(40.840508, -73.85515)",METROPOLITAN AVENUE,0.0,2.0,Failure to Yield Right-of-Way,CAR


In [107]:
precollision['CRASH TIME'] = pd.to_datetime(precollision['CRASH TIME'], format='%H:%M').dt.time
precollision['CRASH TIME INTERVAL'] = precollision['CRASH TIME'].apply(lambda x: f"{x.hour:02}")
precollision.drop(columns=['CRASH TIME'], inplace=True)

precollision['CRASH MOMENT'] = precollision['CRASH TIME INTERVAL'].apply(pre.categorize_moment)

# move TIME INTERVAL to the fourth column
cols = precollision.columns.tolist()
cols = cols[:3] + cols[-1:] + cols[3:-1]
precollision = precollision[cols]

# move CRASH MOMENT to the fifth column
cols = precollision.columns.tolist()
cols = cols[:4] + cols[-1:] + cols[4:-1]
precollision = precollision[cols]

In [108]:
precollision.head(1)

,COLLISION_ID,YEAR,DAY NAME,CRASH MOMENT,CRASH TIME INTERVAL,TYPE OF DAY,CRASH DATE,BOROUGH,ZIP CODE,LOCATION,STREET NAME,NUMBER OF INJURED,NUMBER OF KILLED,CONTRIBUTING FACTOR VEHICLE 1,VEHICLE TYPE CODE 1
0,4345591,2020,Sunday,Night,18,Weekend,2020-09-06 00:00:00+00:00,NaN,NaN,"(40.771038, -73.83413)",WHITESTONE EXPRESSWAY,0.0,2.0,Unsafe Lane Changing,VAN


#### Geographical attributes

At first glance, **ON STREET NAME** and **OFF STREET NAME** seem to be the same attribute, but with different names. The web site of the dataset cointains the following descriptions:

- **ON STREET NAME**: *Street on which the collision occurred*.
- **OFF STREET NAME**: *Street address if known*.

This gives the idea that both attributes probably contain the same information. Furthermore, there are no rows with both attributes filled, which makes the idea of merging both attributes plausible.

In [109]:
collision[(collision['ON STREET NAME'].notnull()) & (collision['OFF STREET NAME'].notnull())].shape

(0, 29)

In [110]:
collision[(collision['ON STREET NAME'].notnull()) | (collision['OFF STREET NAME'].notnull())].shape

(115514, 29)

The resulting attribute after merging both columns is called **STREET NAME** and contains the street name/address where the collision occurred, with no missing values. Some rows will have a more detailed description of the street, while others will only have the name of the street. 

**CROSS STREET**, which is the third attribute related to the street enviroment, and represents the name of the closest street crossing the street of the crash, can be dropped since it is not useful for the intended analysis.

Similarly, **LOCATION** seems to contain the tuple (**LATITUDE**, **LONGITUDE**), so we could, a priori, remove the two extra attributes.

In [111]:
collision[(collision['LOCATION'].notnull()) & (collision['LATITUDE'].notnull()) & (collision['LONGITUDE'].notnull())].shape

(108073, 29)

The number of rows where the three attributes are not missing does not cover the total number of rows, but there are no rows where the **LOCATION** attribute is missing and at least one of the other two attributes is not missing.

In [112]:
collision[(collision['LOCATION'].isnull()) & (collision['LATITUDE'].isnull()) & (collision['LONGITUDE'].notnull()) | (collision['LATITUDE'].notnull()) & (collision['LONGITUDE'].isnull())].shape

(0, 29)

In [113]:
collision[(collision['LOCATION'].isnull())].shape

(7667, 29)

Which makes the rest of the rows (7667) with missing values in the three attributes. This means that the **LATITUDE** and **LONGITUDE** attributes can be removed, since the **LOCATION** attribute contains the same information. With this transformation, the number of attributes is reduced by two.

The clusterization was done with the *key collision* method and the *fingerprint* keying function. No significant variations were found after a copuble iterations in the values of the attributes but misspellings were found. The misspellings were corrected and the clusterization was done again. The results were the same, which means that the values were already consistent. To verify the result, a Neares Neighbours analysis was done as well but without finding any significant variation.

--> mas

#### Vehicle attributes

Regarding vehicle information, the statement of the project specifies that only the **VEHICLE CODE TYPE 1** is of interest to the visualization, so all the other vehicle codes can be removed (2-5). With this, the cotributing factors of the other vehicles can also be removed.

We have seen already that there are many classes of the **VEHICLE CODE TYPE 1** values, so we reduced the number of classes by clusterization.

In [114]:
len(collision['VEHICLE TYPE CODE 1'].unique())

361

With the clusterization (key collision and fingerprint keying function) of the **VEHICLE TYPE CODE 1** attribute we found a lot of misspellings and inconsistencies. The clusterization was done iteratively, correcting the misspellings and inconsistencies found in each iteration. After a couple of iterations, where a lot of subclasses were merged into the same type of vehicle thus aggregating more the data, the number of unique clasess was reduced to the point where the last subclasses manually could be merged manually:

In [115]:
precollision = pre.cluterize_vehicle_type(precollision, 'VEHICLE TYPE CODE 1')

The resulting classes of the **VEHICLE CODE TYPE 1** attribute are:

In [116]:
len(precollision['VEHICLE TYPE CODE 1'].unique()), precollision['VEHICLE TYPE CODE 1'].unique()

(13,
 array(['VAN', 'CAR', 'BUS', 'TRACTOR', 'TAXI', 'SCOOTER', 'TRUCK',
        'MOTORCYCLE', 'BICYCLE', 'OTHERS', 'AMBULANCE', 'FDNY', 'UNKNOWN'],
       dtype=object))

This types of vehicles were selected so there wasn't too much nor too many classes and the user could still get an insight of the analysis.

In [117]:
precollision['VEHICLE TYPE CODE 1'].value_counts()

VEHICLE TYPE CODE 1
CAR           54690
VAN           41651
TRUCK          6684
TAXI           4806
BICYCLE        2083
BUS            1430
MOTORCYCLE     1275
TRACTOR        1098
UNKNOWN         739
AMBULANCE       470
OTHERS          414
SCOOTER         324
FDNY             76
Name: count, dtype: int64

Similar strategy was done with the **CONTRIBUTING FACTOR VEHICLE 1** attribute. However, the aggrgation was not so exhaustive since this attribute wasn't needed a priori for the main questions that the visualizations should answer. For this attribute basic merge transformations were applied in OpenRefine until no "strange" or "uninformative" nor repeated classes remained.

In [118]:
len(precollision['CONTRIBUTING FACTOR VEHICLE 1'].unique()), precollision['CONTRIBUTING FACTOR VEHICLE 1'].unique()

(56,
 array(['Unsafe Lane Changing', 'Driver Inattention/Distraction',
        'Pedestrian/Bicyclist/Other Pedestrian Error/Confusion',
        'Unspecified', 'Failure to Yield Right-of-Way',
        'Turning Improperly', 'Traffic Control Disregarded',
        'Other Vehicular', 'Unsafe Speed',
        'Passing or Lane Usage Improper', 'Driver Inexperience',
        'Reaction to Uninvolved Vehicle', 'Following Too Closely',
        'Aggressive Driving/Road Rage', 'Passing Too Closely',
        'Oversized Vehicle', 'Alcohol Involvement', 'Backing Unsafely',
        'Steering Failure', 'View Obstructed/Limited', 'Fell Asleep',
        'Brakes Defective', 'Pavement Defective', 'Pavement Slippery',
        'Failure to Keep Right', 'Cell Phone (hand-Held)', 'Glare',
        'Passenger Distraction', nan, 'Driverless/Runaway Vehicle',
        'Obstruction/Debris', 'Animals Action',
        'Traffic Control Device Improper/Non-Working',
        'Tire Failure/Inadequate', 'Outside Car Distracti

#### Number of persons attributes

For the visualization purposes, the differentantion of **PERSONS**, **PEDESTRIANS**, **CYCLISTS** and **MOTORISTS** (**INJURED/KILLED**) is irrelevant. A more useful attribute would be the total number of persons involved in the collision. This can be obtained by summing the four attributes under the assumption that the **PERSONS** attribute is not the sum of the other three attributes. 

This condition was needed to be checked because the documentation of the dataset was not precise enough to determinate if **NUMBER OF PERSON INJURED/KILLED** was an aggregate from the other three columns or not.

*Note: The metadata information available in the web of the dataset was: "Number of persons injured/killed" regarding the **NUMBER OF PERSONS INJURED/KILLED**.*

In [119]:
collision['NUMBER OF PERSONS INJURED'].equals(collision['NUMBER OF PEDESTRIANS INJURED'] + collision['NUMBER OF CYCLIST INJURED'] + collision['NUMBER OF MOTORIST INJURED'])

False

In [120]:
collision['NUMBER OF PERSONS INJURED'].equals(collision['NUMBER OF PEDESTRIANS INJURED'])


False

As seen by the logical comprobations, the **NUMBER OF PERSONS INJURED/KILLED** is not the sum of the other three attributes. Furthermore, the terms persons and pedestrians are not equal, as one could have thought that the term persons was used to refer to pedestrians.

Based on this, the discrete attributes refering to the injured people were summed to obtain **NUMBER OF INJURED** and the discrete attributes refering to the killed people were summed to obtain **NUMBER OF KILLED**. The **NUMBER OF INJURED/KILLED** attributes were removed.

#### OpenRefine results

In [121]:
precollision.head()

,COLLISION_ID,YEAR,DAY NAME,CRASH MOMENT,CRASH TIME INTERVAL,TYPE OF DAY,CRASH DATE,BOROUGH,ZIP CODE,LOCATION,STREET NAME,NUMBER OF INJURED,NUMBER OF KILLED,CONTRIBUTING FACTOR VEHICLE 1,VEHICLE TYPE CODE 1
0,4345591,2020,Sunday,Night,18,Weekend,2020-09-06 00:00:00+00:00,NaN,NaN,"(40.771038, -73.83413)",WHITESTONE EXPRESSWAY,0.0,2.0,Unsafe Lane Changing,VAN
1,4459141,2020,Sunday,Morning,09,Weekend,2020-09-20 00:00:00+00:00,NaN,NaN,"(40.722095, -73.77772)",GRAND CENTRAL PKWY,0.0,0.0,Driver Inattention/Distraction,CAR
2,4461437,2020,Thursday,Night,22,Weekday,2020-09-24 00:00:00+00:00,MANHATTAN,10039.0,"(40.824757, -73.94052)",8 AVENUE,0.0,0.0,Pedestrian/Bicyclist/Other Pedestrian Error/Co...,CAR
3,4336560,2020,Thursday,Morning,05,Weekday,2020-08-06 00:00:00+00:00,NaN,NaN,NaN,120 HUGE GRANT CIRCLE,2.0,0.0,Unspecified,CAR
4,4334713,2020,Saturday,Afternoon,17,Weekend,2020-08-01 00:00:00+00:00,BRONX,10462.0,"(40.840508, -73.85515)",METROPOLITAN AVENUE,0.0,2.0,Failure to Yield Right-of-Way,CAR


At this point, the dataset contains the attributes needed (with the *weather* attributes as an exception) for the analysis and some extra attributes that were considered interesting for some possible extra analysis or insights that we could think about.

### Missing values

It has already been mentioned the existence of some missing values. In the previous section, the verification of missing values was done with the ``.isnull()`` method of ``Pandas``. However, this method does not take into account the ``NaN`` values. In order to check the existence of ``NaN`` values, the ``.isna()`` method was used.

In [122]:
comp = (precollision.isnull().sum() == precollision.isna().sum())
comp[comp == False]

Series([], dtype: bool)

As seen previously, all the missing values of the dataset are detected both with ``.isnull()`` and ``.isna()``. After this check, we could group the attributes with missign values in three separeted clusters:
- Geographical attributes
- Injured/Killed attributes
- Vehicle attributes

In [123]:
precollision.isnull().sum()

COLLISION_ID                         0
YEAR                                 0
DAY NAME                             0
CRASH MOMENT                         0
CRASH TIME INTERVAL                  0
TYPE OF DAY                          0
CRASH DATE                           0
BOROUGH                          40671
ZIP CODE                         40686
LOCATION                          7667
STREET NAME                        226
NUMBER OF INJURED                    2
NUMBER OF KILLED                     4
CONTRIBUTING FACTOR VEHICLE 1      351
VEHICLE TYPE CODE 1                  0
dtype: int64

#### Imputation of geographic attributes

The first cluster is formed with reference to the geographicals attributes. The attributes in this cluster are:
- **BOROUGH**
- **ZIP CODE**
- **LOCATION**
- **STREET NAME**
- **CROSS STREET NAME**

Notice that the attributes with the less missing values is **STREET NAME** with only a $0.20\%$ of the entire dataset, partially thanks to the merge of **ON STREET** and **OFF STREET** attributes in the previous sections. We used this information to impute the missing coordinates of the **LOCATION** attribute and the rest of the attributes sequentially.


In [124]:
precollision['LOCATION'].isnull().sum(), precollision['STREET NAME'].isnull().sum()

(7667, 226)

In [125]:
# create a longitude and latitude column from the coordinates column
precollision[['LATITUDE', 'LONGITUDE']] = precollision['LOCATION'].str.split(', ', expand=True)
precollision['LATITUDE'] = precollision['LATITUDE'].str.replace('(', '')
precollision['LONGITUDE'] = precollision['LONGITUDE'].str.replace(')', '')

# convert the columns to float
precollision['LATITUDE'] = precollision['LATITUDE'].astype(float)
precollision['LONGITUDE'] = precollision['LONGITUDE'].astype(float)

precollision.drop(columns=['LOCATION'], inplace=True)

In [126]:
nyc_map = gpd.read_file('Data/new-york-city-boroughs-ny_.geojson')

boroughs = ['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island']

borough_polly = {}
for b in boroughs:
    poly = nyc_map[nyc_map['name'] == b]['geometry']
    borough_polly[b] = poly.values[0]

In [127]:
if not os.path.exists(f'{dir}/precollision_2018-2020_prepro_v2.csv'):
    for idx, row in precollision.iterrows():
        if row['BOROUGH'] is not None:
            lon = row['LONGITUDE']
            lat = row['LATITUDE']

            if lat is not None and lon is not None:
                p = Point(lon, lat)
                for b, poly in borough_polly.items():
                    if p.within(poly):
                        precollision.loc[idx, 'BOROUGH'] = b.upper()
                        break
    
    precollision.to_csv(f'{dir}/precollision_2018-2020_prepro_v2.csv', index=False)
else:
    precollision = pd.read_csv(f'{dir}/precollision_2018-2020_prepro_v2.csv')

c:\Users\jpniz\OneDrive\Documentos\UPC\Q5\.conda\Lib\site-packages\shapely\predicates.py:946: RuntimeWarning: invalid value encountered in within
  return lib.within(a, b, **kwargs)


In [128]:
precollision['ZIP CODE'].isnull().sum(), precollision['BOROUGH'].isnull().sum()

(40686, 6482)

EXPLICAR EL PORQ HAY MENOS MISSING VALUES EN BOROUGH QUE EN LOCATION

In [129]:
precollision[(precollision['LATITUDE'].isna()) & (precollision['BOROUGH'].isna())].shape

(5429, 16)

#### Imputation of vehicle attributes

In [130]:
# poner unespecified en factor $i$ si el vehiculo $i$ es no null y factor $i$ es null
# o
# poner unknown en vehiculo $i$ si el vehiculo $i$ es null y factor $i$ no es null

# son disjuntas estas operaciones?????????????

In some rows of the dataset, the **CONTRIBUTING FACTOR VEHICLE** is missing but the **VEHICLE TYPE CODE** is not. This suggests that the vehicle type is known, but the factor that contributed to the collision is not. In order to fill this missing values, the factor was set as *unespecified*. This was done for all the rows and columns where the **CONTRIBUTING FACTOR VEHICLE** was missing with the above condition.

In [131]:
pre.imputation_with_ref_col(precollision, 'CONTRIBUTING FACTOR VEHICLE 1', 'VEHICLE TYPE CODE 1', 'Unspecified')

Likewise, in some rows of the dataset, the **VEHICLE TYPE CODE** is missing but the **CONTRIBUTING FACTOR VEHICLE** is not. This suggests that the factor that contributed to the collision is known, but the vehicle type is not. In order to fill this missing values, the vehicle type was set as *unknown*. This was done for all the rows and columns where the **VEHICLE TYPE CODE** was missing with the above condition.

In [132]:
pre.imputation_with_ref_col(precollision, 'VEHICLE TYPE CODE 1', 'CONTRIBUTING FACTOR VEHICLE 1', 'UNKNOWN')

In [133]:
precollision.isnull().sum()

COLLISION_ID                         0
YEAR                                 0
DAY NAME                             0
CRASH MOMENT                         0
CRASH TIME INTERVAL                  0
TYPE OF DAY                          0
CRASH DATE                           0
BOROUGH                           6482
ZIP CODE                         40686
STREET NAME                        226
NUMBER OF INJURED                    2
NUMBER OF KILLED                     4
CONTRIBUTING FACTOR VEHICLE 1        0
VEHICLE TYPE CODE 1                  0
LATITUDE                          7667
LONGITUDE                         7667
dtype: int64

<!-- comentar la reduccion de missing values -->

Notice that the only missing values in the **CONTRIBUTING FACTOR VEHICLE** attribute and **VEHICLE TYPE CODE** attribute are in the same rows. This means that the number of missing values in the **CONTRIBUTING FACTOR VEHICLE** attribute and **VEHICLE TYPE CODE** attribute is the same.

In [134]:
# como se hace para imputar los missing de los dos campos a la vez?

# code 1 como unespecified/unknown
# code resto no existent

#### Imputation of number of person attributes

In [135]:
# como se hace para imputar los missing del numero de personas? o mejor se elimina?

In [136]:
precollision.columns

Index(['COLLISION_ID', 'YEAR', 'DAY NAME', 'CRASH MOMENT',
       'CRASH TIME INTERVAL', 'TYPE OF DAY', 'CRASH DATE', 'BOROUGH',
       'ZIP CODE', 'STREET NAME', 'NUMBER OF INJURED', 'NUMBER OF KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'VEHICLE TYPE CODE 1', 'LATITUDE',
       'LONGITUDE'],
      dtype='object')

In [137]:
precollision[precollision['NUMBER OF INJURED'].isnull() | precollision['NUMBER OF KILLED'].isnull()]

,COLLISION_ID,YEAR,DAY NAME,CRASH MOMENT,CRASH TIME INTERVAL,TYPE OF DAY,CRASH DATE,BOROUGH,ZIP CODE,STREET NAME,NUMBER OF INJURED,NUMBER OF KILLED,CONTRIBUTING FACTOR VEHICLE 1,VEHICLE TYPE CODE 1,LATITUDE,LONGITUDE
42087,4026403,2018,Sunday,Morning,06,Weekend,2018-09-30 00:00:00+00:00,QUEENS,11368.0,102-21 ROOSEVELT AVENUE,NaN,NaN,Unspecified,TAXI,40.749767,-73.86381
62406,4026219,2018,Wednesday,Afternoon,13,Weekday,2018-08-08 00:00:00+00:00,STATEN ISLAND,10312.0,1531 ARTHUR KILL ROAD,2.0,NaN,Lost Consciousness,CAR,40.562523,-74.19211
69596,4026267,2018,Thursday,Morning,08,Weekday,2018-08-23 00:00:00+00:00,MANHATTAN,10001.0,135 WEST 26 STREET,0.0,NaN,Driver Inattention/Distraction,CAR,40.745280,-73.99254
90894,4026185,2018,Monday,Afternoon,12,Weekday,2018-07-23 00:00:00+00:00,BROOKLYN,11221.0,STUYVESANT AVENUE,NaN,0.0,Unspecified,CAR,40.686030,-73.93268
114639,4026046,2018,Tuesday,Morning,05,Weekday,2018-06-05 00:00:00+00:00,BRONX,10459.0,1145 WEST FARMS ROAD,2.0,NaN,Unspecified,VAN,40.826992,-73.88984


Since the resulting rows with missing values of the number of persons involved in the collision are a small fraction of the total dataset, we decided to set them to 0, assuming that there were no persons involved in the collision.

Given the small percentage of the rows $0.01\%$, it was considered that there would not be a significant impact in the final visualization whether the missing values were set to 0 or the rows with missing values were dropped. However, we decided to set the missing values to 0 in order to keep the rows and not lose some of its information. This was done also because **NUMBER OF INJURED** and  **NUMBER OF KILLED** were attributes that are not necessary for the main visualizations and we were only keeping them in case we found an intereseting extra visualization with them.

In [138]:
precollision['NUMBER OF INJURED'].fillna(0, inplace=True)
precollision['NUMBER OF KILLED'].fillna(0, inplace=True)

With this done, the only meaningful missing values are the 226 of **STREET NAME**, that is because the other location related attributes could be mostly imputed with **STREET NAME**.

In [139]:
precollision.isnull().sum()

COLLISION_ID                         0
YEAR                                 0
DAY NAME                             0
CRASH MOMENT                         0
CRASH TIME INTERVAL                  0
TYPE OF DAY                          0
CRASH DATE                           0
BOROUGH                           6482
ZIP CODE                         40686
STREET NAME                        226
NUMBER OF INJURED                    0
NUMBER OF KILLED                     0
CONTRIBUTING FACTOR VEHICLE 1        0
VEHICLE TYPE CODE 1                  0
LATITUDE                          7667
LONGITUDE                         7667
dtype: int64

### Save the results

In [140]:
if not os.path.exists(f'{dir}/collisions_clean.csv'):
    precollision.to_csv(f'{dir}/collisions_clean.csv', index=False)

In [141]:
precollision.columns

Index(['COLLISION_ID', 'YEAR', 'DAY NAME', 'CRASH MOMENT',
       'CRASH TIME INTERVAL', 'TYPE OF DAY', 'CRASH DATE', 'BOROUGH',
       'ZIP CODE', 'STREET NAME', 'NUMBER OF INJURED', 'NUMBER OF KILLED',
       'CONTRIBUTING FACTOR VEHICLE 1', 'VEHICLE TYPE CODE 1', 'LATITUDE',
       'LONGITUDE'],
      dtype='object')